In [28]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd

In [29]:
data = vbt.BinanceData.fetch('BTCUSDT', start="2021-01")

0it [00:00, ?it/s]

In [30]:
data.data['BTCUSDT'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 828 entries, 2021-01-01 00:00:00+00:00 to 2023-04-08 00:00:00+00:00
Freq: D
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                828 non-null    float64
 1   High                828 non-null    float64
 2   Low                 828 non-null    float64
 3   Close               828 non-null    float64
 4   Volume              828 non-null    float64
 5   Quote volume        828 non-null    float64
 6   Trade count         828 non-null    int32  
 7   Taker base volume   828 non-null    float64
 8   Taker quote volume  828 non-null    float64
dtypes: float64(8), int32(1)
memory usage: 61.5 KB


In [31]:
data.plot().show() 

In [32]:
high_price = data.high
low_price = data.low
close_price = data.close
open_price = data.open

In [33]:
def change(series, lenght=1):
    return (series - series.shift(lenght))

def calculate_plus_dm(upMove, downMove):
    plusDM = pd.Series(np.nan, index=upMove.index)  # Initialize the result Series with NaN values
    positive_move = (upMove > downMove) & (upMove > 0)  # Check if upMove is greater than downMove and greater than 0
    plusDM[positive_move] = upMove[positive_move]  # Assign the positive price movement to plusDM where the condition is true
    plusDM = plusDM.fillna(0)  # Fill NaN values with 0
    
    return plusDM

def calculate_minus_dm(upMove, downMove):
    minusDM = pd.Series(np.nan, index=downMove.index)  # Initialize the result Series with NaN values
    negative_move = (downMove > upMove) & (downMove > 0)  # Check if downMove is greater than upMove and greater than 0
    minusDM[negative_move] = downMove[negative_move]  # Assign the negative price movement to minusDM where the condition is true
    minusDM = minusDM.fillna(0)  # Fill NaN values with 0
    
    return minusDM

In [34]:
upMove = change(high_price, lenght=1)
downMove = - change(low_price, lenght=1)

In [35]:
plusDM = calculate_plus_dm(upMove, downMove)
minusDM = calculate_minus_dm(upMove, downMove)

In [36]:
tr = vbt.pandas_ta("TRUE_RANGE").run(data.high, data.low, data.close)
trur = vbt.pandas_ta("RMA").run(tr.truerange, length=14)

In [37]:
rma_plusDM = vbt.pandas_ta("RMA").run(plusDM, length=14)
rma_minusDM = vbt.pandas_ta("RMA").run(minusDM, length=14) 

plusDI = 100 * rma_plusDM.rma / trur.rma
minusDI = 100 * rma_minusDM.rma / trur.rma

dx_rma = vbt.pandas_ta("RMA").run(abs(plusDI - minusDI) / (plusDI + minusDI), length=14)
dx = 100 * dx_rma.rma

In [38]:
adx = vbt.pandas_ta("RMA").run(dx, length=14)

In [39]:
adx_level = 25
entries_long = (plusDI > minusDI) & (plusDI > adx_level)
exits_long = (plusDI < minusDI)
exits_long_2 = (plusDI < adx_level)

entries_short = (minusDI > plusDI) & (minusDI > adx_level)
exits_short = (minusDI < plusDI)
exits_short = (minusDI < adx_level)

In [40]:
start_date = "2022-01-01"
stop_date = "2023-04-01"
mask = (close_price.index >= start_date) & (close_price.index <= stop_date)

In [49]:
pf = vbt.Portfolio.from_signals(
    close=close_price.loc[mask], 
    entries=entries_long.loc[mask],
    exits=exits_long.loc[mask],
    short_entries=entries_short.loc[mask],
    short_exits=exits_short.loc[mask],
    size=100,
    size_type='value',
    init_cash=4000,
    from_ago=1,
    tp_stop=0.02,
    sl_stop=0.01,
    open=open_price.loc[mask],
    high=high_price.loc[mask],
    low=low_price.loc[mask],
    # price=vbt.Param(["close", "nextopen"]),
)
# entries_long.vbt.signals.plot_as_entries(adx.rma, fig=fig)
# exits_long.vbt.signals.plot_as_exits(adx.rma, fig=fig)

In [50]:
pf.stats()

Start                         2022-01-01 00:00:00+00:00
End                           2023-04-01 00:00:00+00:00
Period                                456 days 00:00:00
Start Value                                      4000.0
Min Value                                   3965.601247
Max Value                                        4003.0
End Value                                   3974.162025
Total Return [%]                              -0.645949
Benchmark Return [%]                         -40.378981
Total Time Exposure [%]                       34.868421
Max Gross Exposure [%]                         2.556924
Max Drawdown [%]                               0.934268
Max Drawdown Duration                 436 days 00:00:00
Total Orders                                        275
Total Fees Paid                                     0.0
Total Trades                                        138
Win Rate [%]                                  27.737226
Best Trade [%]                                  

In [51]:
pf.plot(settings=dict(bm_returns=False)).show()

In [20]:
pf.orders.records_readable

,Order Id,Column,Signal Index,Creation Index,Fill Index,Size,Price,Fees,Side,Type,Stop Type
0,0,0,2022-01-01 00:00:00+00:00,2022-01-02 00:00:00+00:00,2022-01-02 00:00:00+00:00,0.084591,47286.18,0.0,Sell,Market,None
1,1,0,2022-01-20 00:00:00+00:00,2022-01-21 00:00:00+00:00,2022-01-21 00:00:00+00:00,0.084591,36445.31,0.0,Buy,Market,None
2,2,0,2022-01-21 00:00:00+00:00,2022-01-22 00:00:00+00:00,2022-01-22 00:00:00+00:00,0.140201,35071.42,0.0,Sell,Market,None
3,3,0,2022-02-04 00:00:00+00:00,2022-02-05 00:00:00+00:00,2022-02-05 00:00:00+00:00,0.140201,41382.59,0.0,Buy,Market,None
4,4,0,2022-02-06 00:00:00+00:00,2022-02-07 00:00:00+00:00,2022-02-07 00:00:00+00:00,0.091976,43839.99,0.0,Buy,Market,None
5,5,0,2022-02-17 00:00:00+00:00,2022-02-18 00:00:00+00:00,2022-02-18 00:00:00+00:00,0.183951,39974.44,0.0,Sell,Market,None
6,6,0,2022-02-28 00:00:00+00:00,2022-03-01 00:00:00+00:00,2022-03-01 00:00:00+00:00,0.091976,44421.20,0.0,Buy,Market,None
7,7,0,2022-03-04 00:00:00+00:00,2022-03-05 00:00:00+00:00,2022-03-05 00:00:00+00:00,0.082940,39397.96,0.0,Sell,Market,None
8,8,0,2022-03-10 00:00:00+00:00,2022-03-11 00:00:00+00:00,2022-03-11 00:00:00+00:00,0.082940,38729.57,0.0,Buy,Market,None
9,9,0,2022-03-24 00:00:00+00:00,2022-03-25 00:00:00+00:00,2022-03-25 00:00:00+00:00,0.074992,44313.16,0.0,Buy,Market,None


In [125]:
xx = vbt.PF.from_random_signals(
    vbt.YFData.fetch("BTC-USD", start="2021-01", end="2021-02"), 
    n=3, 
    price=vbt.Param(["close", "nextopen"])
)

BTC-USD: No data found for this date range, symbol may be delisted


C:\Users\paica\AppData\Local\Temp\ipykernel_31872\3905331103.py:2: UserWarning:

Symbol 'BTC-USD' returned an empty array. Skipping.



ValueError: No symbols could be fetched